### Задачи к Лекции 4

__Исходные данные__

Дан файл **"mlbootcamp5_train.csv"**. В нем содержатся данные об опросе 70000 пациентов с целью определения наличия заболеваний сердечно-сосудистой системы (ССЗ). Данные в файле промаркированы и если у человека имееются ССЗ, то значение **cardio** будет равно 1, в противном случае - 0. Описание и значения полей представлены во второй лекции.

__Загрузка файла__

In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import seaborn as sns
import sklearn
from matplotlib import pyplot as plt
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [10, 5]


df = pd.read_csv("/content/mlbootcamp5_train.csv",
                 sep=";",
                 index_col="id")
# Делаем one-hot кодирование
chol = pd.get_dummies(df["cholesterol"], prefix="chol")
gluc = pd.get_dummies(df["gluc"], prefix="gluc")
df = pd.concat([df, chol, gluc], axis=1)

# Делаем пол бинарным признаком
df["gender_bin"] = df["gender"].map({1: 0, 2: 1})
df.head()

,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio,chol_1,chol_2,chol_3,gluc_1,gluc_2,gluc_3,gender_bin
id,,,,,,,,,,,,,,,,,,,
0,18393,2,168,62.0,110,80,1,1,0,0,1,0.0,True,False,False,True,False,False,1
1,20228,1,156,85.0,140,90,3,1,0,0,1,1.0,False,False,True,True,False,False,0
2,18857,1,165,64.0,130,70,3,1,0,0,0,1.0,False,False,True,True,False,False,0
3,17623,2,169,82.0,150,100,1,1,0,0,1,1.0,True,False,False,True,False,False,1
4,17474,1,156,56.0,100,60,1,1,0,0,0,0.0,True,False,False,True,False,False,0


## Задачи

__1. Хоть в sklearn и присутствует реализация метода k-ближайших соседей, я же предлагаю попробовать вам написать его самостоятельно.__

* __создать классификатор используя только pandas, numpy и scipy. Гиперпараметром данного классификатора должно быть число ближайших соседей. (Необязательно) можно добавить метрику расстояния и выбор весов.__
* __С помощью кросс-валидации найти оптимальное количество ближайших соседей и (необязательно) набор признаков.__

Алгоритм работы классификатора:
 1. Для заданного прецедент  $\vec{x}$ мы считаем расстояние до всех прецедентов в обучающей выборке.
 2. Сортируем прецеденты по расстоянию до $\vec{x}$.
 3. Отбираем $k$ минимальных значений
 4. Устраиваем голосование между отобранными прецедент.

In [2]:
import numpy as np
import pandas as pd
from scipy.spatial.distance import cdist
from collections import Counter
from sklearn.model_selection import train_test_split

In [3]:
class KNNClassifier:
    def __init__(self, n_neighbors):    # init - инициализация
        self.n_neighbors = n_neighbors  # Гиперпараметр k
        self.X_train = None
        self.y_train = None

    def fit(self, X, y):  # Сохраняет обучающие данные
        self.X_train = np.array(X) # Преобразует их в numpy массивы для удобства вычислений и сохраняет в атрибутах класса.
        self.y_train = np.array(y)

    def predict(self, X): # Предсказывает класс для новых данных
        X = np.array(X)
        predictions = []
        for x in X:
            # 1. Вычисляем расстояния от x до всех точек в X_train
            distances = cdist([x], self.X_train, metric='euclidean')[0]  #cdist из scipy.spatial.distance вычисляет расстояния между текущей точкой x и всеми точками обучающей выборки
                                    # используется Евклидова метрика -  стандартный способ измерения расстояния между двумя точками в многомерном пространстве.

            # 2. Находим k ближайших соседей
            nearest_indices = np.argsort(distances)[:self.n_neighbors] # возвращает индексы точек, отсортированных по расстоянию (от ближайшей к самой далёкой).
            nearest_labels = self.y_train[nearest_indices] # получает метки этих ближайших соседей.


            # 3. Выбираем наиболее частый класс
            most_common = Counter(nearest_labels).most_common(1)[0][0] #подсчитывает количество каждого класса среди соседей.
            predictions.append(most_common)
        return np.array(predictions)

    def score(self, X, y): # Вычисляет точность (accuracy)
        y_pred = self.predict(X)
        return np.mean(y_pred == y)

In [24]:
# Загрузка данных (предположим, что данные уже в DataFrame df)
X = df[['age', 'gender', 'height', 'weight', 'ap_hi', 'ap_lo']].values
y = df['cardio'].values

# Разделение на train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [12]:
# Создаем классификатор с k=5 и евклидовой метрикой
knn = KNNClassifier(n_neighbors=5)
knn.fit(X_train, y_train)

# Предсказание и оценка
y_pred = knn.predict(X_test)
accuracy = knn.score(X_test, y_test)
print(f"Точность: {accuracy:.3f}")

Точность: 0.682


**Комментарии:** Ваши комментарии здесь.

**2. Определить какой из трех классификаторов (kNN, наивный Байес, решающее дерево) лучший в каждой метрике по отдельности: accuracy, F1-мера, ROC AUC, функция потерь. Использовать набор признаков: 'age', 'weight', 'height', 'ap_lo', 'ap_hi'.**

**(Необязательно) Найти оптимальный набор признаков.**

In [29]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier

# Инициализация моделей
knn = KNeighborsClassifier(n_neighbors=5)
nb = GaussianNB()
dt = DecisionTreeClassifier(max_depth=3, random_state=42)

In [27]:
import numpy as np

# Проверка на NaN в y_train
nan_indices = np.isnan(y_train)
print(f"Найдено NaN в y_train: {np.sum(nan_indices)}")

# Удаление NaN (если они есть)
if np.any(nan_indices):
    y_train1 = y_train[~nan_indices]
    X_train1 = X_train[~nan_indices]  # Не забудьте удалить соответствующие строки в X_train!

Найдено NaN в y_train: 1


In [30]:
# Обучение наивного Байеса и решающего дерева
knn.fit(X_train1, y_train1)
nb.fit(X_train1, y_train1)
dt.fit(X_train1, y_train1)

DecisionTreeClassifier(max_depth=3, random_state=42)

**1. Метрика Accuracy**

Измеряет долю правильных предсказаний среди всех предсказаний.

In [31]:
#Результаты по метрике Точности для кажкого классификатора
from sklearn.metrics import accuracy_score

knn_acc = accuracy_score(y_test, knn.predict(X_test))
nb_acc = accuracy_score(y_test, nb.predict(X_test))
dt_acc = accuracy_score(y_test, dt.predict(X_test))

print(f"kNN Accuracy: {knn_acc:.3f}")
print(f"Naive Bayes Accuracy: {nb_acc:.3f}")
print(f"Decision Tree Accuracy: {dt_acc:.3f}")

kNN Accuracy: 0.682
Naive Bayes Accuracy: 0.538
Decision Tree Accuracy: 0.724


**2. Метрика F1-мера (среднее гармоническое)**



Измеряет гармоническое среднее между Precision (точность) и Recall (полнота).

In [32]:
from sklearn.metrics import f1_score

knn_f1 = f1_score(y_test, knn.predict(X_test))
nb_f1 = f1_score(y_test, nb.predict(X_test))
dt_f1 = f1_score(y_test, dt.predict(X_test))

print(f"kNN F1: {knn_f1:.3f}")
print(f"Naive Bayes F1: {nb_f1:.3f}")
print(f"Decision Tree F1: {dt_f1:.3f}")

kNN F1: 0.676
Naive Bayes F1: 0.168
Decision Tree F1: 0.715


**3. Метрика ROC AUC**

ROC AUC - это площадь (Area Under Curve) под кривой ошибок (Receiver Operating Characteristic curve).

Кривая ошибок - это линия от (0,0) до (1,1) в координатах True Positive Rate(TPR) и False Positive Rate(FRP)

Измеряет способность модели разделять классы

In [33]:
from sklearn.metrics import roc_auc_score

knn_auc = roc_auc_score(y_test, knn.predict_proba(X_test)[:, 1])
nb_auc = roc_auc_score(y_test, nb.predict_proba(X_test)[:, 1])
dt_auc = roc_auc_score(y_test, dt.predict_proba(X_test)[:, 1])

print(f"kNN AUC: {knn_auc:.3f}")
print(f"Naive Bayes AUC: {nb_auc:.3f}")
print(f"Decision Tree AUC: {dt_auc:.3f}")

kNN AUC: 0.727
Naive Bayes AUC: 0.731
Decision Tree AUC: 0.783


**4. Функия потерь**

Измеряет штраф за:

*   Предсказание низкой вероятности для правильного класса.

*   Предсказание высокой вероятности для неправильного класса.

In [34]:
from sklearn.metrics import log_loss

knn_loss = log_loss(y_test, knn.predict_proba(X_test))
nb_loss = log_loss(y_test, nb.predict_proba(X_test))
dt_loss = log_loss(y_test, dt.predict_proba(X_test))

print(f"kNN Log Loss: {knn_loss:.3f}")
print(f"Naive Bayes Log Loss: {nb_loss:.3f}")
print(f"Decision Tree Log Loss: {dt_loss:.3f}")

kNN Log Loss: 2.324
Naive Bayes Log Loss: 1.219
Decision Tree Log Loss: 0.556


In [35]:
results = {
    'kNN': {
        'Accuracy': knn_acc,
        'F1': knn_f1,
        'ROC AUC': knn_auc,
        'Log Loss': knn_loss
    },
    'Naive Bayes': {
        'Accuracy': nb_acc,
        'F1': nb_f1,
        'ROC AUC': nb_auc,
        'Log Loss': nb_loss
    },
    'Decision Tree': {
        'Accuracy': dt_acc,
        'F1': dt_f1,
        'ROC AUC': dt_auc,
        'Log Loss': dt_loss
    }
}

In [36]:
results_df = pd.DataFrame(results).T  # Транспонируем для удобства

In [39]:
results_df

,Accuracy,F1,ROC AUC,Log Loss
kNN,0.681900,0.676263,0.727120,2.324488
Naive Bayes,0.537528,0.167827,0.731005,1.218953
Decision Tree,0.724198,0.715407,0.782605,0.556252


**Комментарии:** лучше со всем справилось дерево решений.